In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

2023-11-07 10:59:21,659 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.sql.functions import *

In [5]:
adv_path="file:///home/labuser/Downloads/43801857-7a5c-4d68-a1cc-3bd11a166912_83d04ac6-cb74-4a96-a06a-e0d5442aa126_advertising_spend.csv"
prod_path="file:///home/labuser/Downloads/53b473d6-8cfc-4518-88a7-02da0a690b68_83d04ac6-cb74-4a96-a06a-e0d5442aa126_products.csv"
sales_path="file:///home/labuser/Downloads/6650c551-5f2f-45b7-a9e7-d43d82422435_83d04ac6-cb74-4a96-a06a-e0d5442aa126_sales.csv"
feedback_path="file:///home/labuser/Downloads/197e029f-9cda-4fe7-bc6b-c6393382d61e_83d04ac6-cb74-4a96-a06a-e0d5442aa126_feedback.csv"
cust1_path="file:///home/labuser/Downloads/a80d3120-2e0f-49cb-970f-4e565587f45b_83d04ac6-cb74-4a96-a06a-e0d5442aa126_customers-1.csv"
cust2_path="file:///home/labuser/Downloads/71de6941-e785-4896-a297-a4ce99d26bf1_83d04ac6-cb74-4a96-a06a-e0d5442aa126_customers-2.csv"



In [7]:
adv_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(adv_path)
products_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(prod_path)
sales_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(sales_path)
feedback_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(feedback_path)
cust1_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(cust1_path)
cust2_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(cust2_path)

In [16]:
products_df.show()
adv_df.show()
sales_df.show()

+---------+--------------------+-----+
|ProductID|         ProductName|Price|
+---------+--------------------+-----+
|        1|     Apple iPhone 13|  799|
|        2|Sony WH-1000XM4 H...|  278|
|        3|"Samsung 55"" QLE...| 1200|
|        4|Apple Watch Series 7|  399|
|        5|JBL Charge 5 Blue...|  179|
|        6|Apple iPad Pro 11...|  749|
+---------+--------------------+-----+

+---------+-----------+-----+----+-----------+
|ChannelID|ChannelName|Month|Year|AmountSpent|
+---------+-----------+-----+----+-----------+
|        1| Google Ads|    1|2021|      12648|
|        2|    Youtube|    1|2021|       6216|
|        3|   Facebook|    1|2021|      11283|
|        4|  Instagram|    1|2021|      10327|
|        5|   LinkedIn|    1|2021|      11561|
|        1| Google Ads|    2|2021|      18610|
|        2|    Youtube|    2|2021|      16135|
|        3|   Facebook|    2|2021|       9449|
|        4|  Instagram|    2|2021|      11805|
|        5|   LinkedIn|    2|2021|      11542

In [33]:
report0=sales_df.join(products_df,sales_df["ProductID"]==products_df["ProductID"],"inner").select("ProductName","quantity","Price")\
        .selectExpr("ProductName","(quantity * Price) as amount")
report0.selectExpr("sum(amount)").show()

+-----------+
|sum(amount)|
+-----------+
|    6069667|
+-----------+



In [35]:
report1=sales_df.join(products_df,sales_df["ProductID"]==products_df["ProductID"],"inner").select("ProductName","quantity","Price")\
        .selectExpr("ProductName","(quantity * Price) as amount").groupBy("ProductName").agg(sum("amount").alias("amount"))\
        .orderBy(col("amount").desc())
report1.show(3)

+--------------------+-------+
|         ProductName| amount|
+--------------------+-------+
|     Apple iPhone 13|2249185|
|"Samsung 55"" QLE...|1486800|
|Apple iPad Pro 11...| 918274|
+--------------------+-------+
only showing top 3 rows



In [43]:
report2=adv_df.groupBy("ChannelName").agg(sum("AmountSpent").alias("total_spent")).orderBy(col("total_spent").desc())
report2.show(1)

+-----------+-----------+
|ChannelName|total_spent|
+-----------+-----------+
| Google Ads|     327598|
+-----------+-----------+
only showing top 1 row



In [47]:
sales_df.groupBy("ProductID","CustomerID").agg(avg("Quantity")).orderBy(col("ProductID").desc()).show()

+---------+----------+-------------+
|ProductID|CustomerID|avg(Quantity)|
+---------+----------+-------------+
|        6|      3770|          2.0|
|        6|      3216|          3.0|
|        6|      4043|          2.0|
|        6|      4926|          3.0|
|        6|      4932|          1.0|
|        6|      2587|          2.0|
|        6|      1506|          2.0|
|        6|      2725|          1.0|
|        6|      1873|          3.0|
|        6|      1741|          3.0|
|        6|      3648|          4.0|
|        6|      4660|          3.0|
|        6|      1184|          2.0|
|        6|       508|          3.0|
|        6|      3271|          1.0|
|        6|      2335|          1.0|
|        6|      2280|          3.0|
|        6|      4219|          2.0|
|        6|      3913|          1.0|
|        6|      1572|          3.0|
+---------+----------+-------------+
only showing top 20 rows



In [66]:
b1=sales_df.groupBy("ProductID").agg(sum("quantity"))
b2=sales_df.join(products_df,["ProductID"],"inner")\
                .selectExpr("ProductID","(quantity * Price) as sales_amount")\
                .groupBy("ProductID").agg(sum("sales_amount"))
b3=b1.join(b2,["ProductID"])
b3.show()

+---------+-------------+-----------------+
|ProductID|sum(quantity)|sum(sales_amount)|
+---------+-------------+-----------------+
|        1|         2815|          2249185|
|        6|         1226|           918274|
|        3|         1239|          1486800|
|        5|         1161|           207819|
|        4|         1153|           460047|
|        2|         2689|           747542|
+---------+-------------+-----------------+



In [67]:
result_df = b3.withColumn(
    "discounted_total_sales",
    when(col("sum(quantity)") > 100, col("sum(sales_amount)") * 0.9).otherwise(col("sum(sales_amount)"))
).show()

+---------+-------------+-----------------+----------------------+
|ProductID|sum(quantity)|sum(sales_amount)|discounted_total_sales|
+---------+-------------+-----------------+----------------------+
|        1|         2815|          2249185|             2024266.5|
|        6|         1226|           918274|              826446.6|
|        3|         1239|          1486800|             1338120.0|
|        5|         1161|           207819|              187037.1|
|        4|         1153|           460047|              414042.3|
|        2|         2689|           747542|              672787.8|
+---------+-------------+-----------------+----------------------+



In [72]:
sales_df.show()

+-------+----------+--------+---------+---------+----------+
|SalesID|      Date|Quantity|ChannelID|ProductID|CustomerID|
+-------+----------+--------+---------+---------+----------+
|      1|2021-06-01|       1|        3|        1|         1|
|      2|2022-05-02|       2|        1|        1|         2|
|      3|2022-03-04|       2|        4|        1|         3|
|      4|2022-06-24|       1|        4|        2|         4|
|      5|2021-12-26|       4|        1|        1|         5|
|      6|2022-08-04|       1|        2|        1|         6|
|      7|2021-10-03|       1|        1|        1|         7|
|      8|2021-08-27|       2|        2|        6|         8|
|      9|2021-01-27|       2|        1|        1|         9|
|     10|2022-07-04|       3|        2|        1|        10|
|     11|2022-05-17|       2|        2|        6|        11|
|     12|2021-03-15|       2|        3|        3|        12|
|     13|2022-10-24|       3|        2|        1|        13|
|     14|2021-12-10|    

In [75]:
sales_df.withColumn("isweekday",dayofweek(col("Date").cast("Date"))).show()

+-------+----------+--------+---------+---------+----------+---------+
|SalesID|      Date|Quantity|ChannelID|ProductID|CustomerID|isweekday|
+-------+----------+--------+---------+---------+----------+---------+
|      1|2021-06-01|       1|        3|        1|         1|        3|
|      2|2022-05-02|       2|        1|        1|         2|        2|
|      3|2022-03-04|       2|        4|        1|         3|        6|
|      4|2022-06-24|       1|        4|        2|         4|        6|
|      5|2021-12-26|       4|        1|        1|         5|        1|
|      6|2022-08-04|       1|        2|        1|         6|        5|
|      7|2021-10-03|       1|        1|        1|         7|        1|
|      8|2021-08-27|       2|        2|        6|         8|        6|
|      9|2021-01-27|       2|        1|        1|         9|        4|
|     10|2022-07-04|       3|        2|        1|        10|        2|
|     11|2022-05-17|       2|        2|        6|        11|        3|
|     

In [76]:
sales_df=sales_df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

sales_df=sales_df.withColumn("DayOfWeek", date_format(col("Date"), "E")) \
                     .withColumn("Category", when(col("DayOfWeek").isin("Sat", "Sun"), "Weekend").otherwise("Weekday"))

# Calculate the total sales revenue for weekends and weekdays
total_sales_revenue = sales_df.groupBy("Category") \
                               .agg(sum("SalesAmount").alias("TotalSalesRevenue")).show()


AnalysisException: cannot resolve '`SalesAmount`' given input columns: [Category, ChannelID, CustomerID, Date, DayOfWeek, ProductID, Quantity, SalesID];
'Aggregate [Category#1963], [Category#1963, sum('SalesAmount) AS TotalSalesRevenue#1981]
+- Project [SalesID#64, Date#1948, Quantity#66, ChannelID#67, ProductID#68, CustomerID#69, DayOfWeek#1955, CASE WHEN DayOfWeek#1955 IN (Sat,Sun) THEN Weekend ELSE Weekday END AS Category#1963]
   +- Project [SalesID#64, Date#1948, Quantity#66, ChannelID#67, ProductID#68, CustomerID#69, date_format(cast(Date#1948 as timestamp), E, Some(UTC)) AS DayOfWeek#1955]
      +- Project [SalesID#64, to_date('Date, Some(yyyy-MM-dd)) AS Date#1948, Quantity#66, ChannelID#67, ProductID#68, CustomerID#69]
         +- Relation[SalesID#64,Date#65,Quantity#66,ChannelID#67,ProductID#68,CustomerID#69] csv
